In [3]:
import numpy as np
import scipy
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
mat_file_list= glob('../../data/Mix_artifacts/raw/*.mat')
csv_file_list=[f.replace('.mat', '.csv') for f in mat_file_list]
csv_file_list=[f.replace('/raw/', '/csv/') for f in csv_file_list]
for i in tqdm(range(len(mat_file_list))):
    mat_file=scipy.io.loadmat(mat_file_list[i])
    value_array=mat_file['signals'][0][0]
    Type_1=value_array[5][0]
    Type_2=value_array[5][1]
    Type_3=value_array[5][2]
    Type_4=value_array[5][3]
    Type_5=value_array[5][4]
    raw_PPG=value_array[1][0]
    PPG=np.zeros([len(Type_1)],dtype=np.int8)
    prev_index=0
    for j in range(len(raw_PPG)):
        next_index=int(len(Type_1)/len(raw_PPG)*(j+1))
        PPG[prev_index:next_index+1]=raw_PPG[j]
        prev_index=int(len(Type_1)/len(raw_PPG)*(j+1))+1
    data = {'PPG':PPG,'Type_1' : Type_1,'Type_2' : Type_2,'Type_3' : Type_3,'Type_4' : Type_4,'Type_5' : Type_5}
    data=pd.DataFrame(data)
    data.to_csv(csv_file_list[i],mode = 'w', index=False)

In [6]:
file_count=0
negative_count=0
pulse=3000
waveform_binary_path='../../data/Mix_artifacts/binary/'
train_binary_df=pd.DataFrame(columns=['FileName','PPG'])
test_binary_df=pd.DataFrame(columns=['FileName','PPG'])
waveform_mean_path='../../data/Mix_artifacts/ratio/'
train_mean_df=pd.DataFrame(columns=['FileName','PPG'])
test_mean_df=pd.DataFrame(columns=['FileName','PPG'])
def csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_path,data_set_class,count):
    data = {'PPG Class':PPG,'PPG' : Type_1}
    data_1=pd.DataFrame(data)
    data_1.to_csv(waveform_path+data_set_class+'/Type_1_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_2}
    data_2=pd.DataFrame(data)
    data_2.to_csv(waveform_path+data_set_class+'/Type_2_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_3}
    data_3=pd.DataFrame(data)
    data_3.to_csv(waveform_path+data_set_class+'/Type_3_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_4}
    data_4=pd.DataFrame(data)
    data_4.to_csv(waveform_path+data_set_class+'/Type_4_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_5}
    data_5=pd.DataFrame(data)
    data_5.to_csv(waveform_path+data_set_class+'/Type_5_'+str(count)+'.csv',mode = 'w', index=False)
    
    
for i in tqdm(range(len(csv_file_list))):
    csv_data=pd.read_csv(csv_file_list[i])
    count=len(csv_data)
    a=np.random.randint(0, 10)
    if a==0 or a==1:
        data_set_class='test'
    else: 
        data_set_class='train'
    for j in range(count//(pulse-500)-1):
        sec_data=csv_data[j*(pulse-500):j*(pulse-500)+pulse]
        PPG=sec_data['PPG'].to_numpy()
        PPG_mean=PPG.mean()
        Type_1=sec_data['Type_1'].to_numpy()
        Type_2=sec_data['Type_2'].to_numpy()
        Type_3=sec_data['Type_3'].to_numpy()
        Type_4=sec_data['Type_4'].to_numpy()
        Type_5=sec_data['Type_5'].to_numpy()
        file_name_list=[]
        PPG_binary_list=[]
        PPG_mean_list=[]
        
        for k in range(1,6):
            file_name_list.append('Type_'+str(k)+'_'+str(count)+'.csv')
            PPG_mean_list.append(PPG_mean)
            if PPG_mean>0:
                PPG_binary_list.append(1)
            else: 
                PPG_binary_list.append(0)
                
        if data_set_class=='train':
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_binary_path,data_set_class,file_count)
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_mean_path,data_set_class,file_count)
            binary_data={'FileName':file_name_list,'PPG':PPG_binary_list}
            mean_data={'FileName':file_name_list,'PPG':PPG_mean_list}
            PPG_binary_temp=pd.DataFrame(binary_data)
            PPG_mean_temp=pd.DataFrame(mean_data)
            train_binary_df = pd.concat([train_binary_df,PPG_binary_temp])
            train_mean_df = pd.concat([train_mean_df,PPG_mean_temp])
            file_count+=1
            
        if data_set_class=='test':
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_binary_path,data_set_class,file_count)
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_mean_path,data_set_class,file_count)
            binary_data={'FileName':file_name_list,'PPG':PPG_binary_list}
            mean_data={'FileName':file_name_list,'PPG':PPG_mean_list}
            PPG_binary_temp=pd.DataFrame(binary_data)
            PPG_mean_temp=pd.DataFrame(mean_data)
            test_binary_df = pd.concat([test_binary_df,PPG_binary_temp])
            test_mean_df = pd.concat([test_mean_df,PPG_mean_temp])
            file_count+=1
train_binary_df.to_csv(waveform_binary_path+'train.csv')
test_binary_df.to_csv(waveform_binary_path+'test.csv')
train_mean_df.to_csv(waveform_mean_path+'train.csv')
test_mean_df.to_csv(waveform_mean_path+'test.csv')

  0%|          | 0/901 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
{}